# JS with HTML

In [1]:
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots

import numpy as np
import webbrowser
import os.path
import re 

def calculateMetric(dfLocal,dfGlobal,option):
  unit = dfLocal['unit'].unique()[0]
  if option == 'progress':
    if unit=='chf' or unit=='amount':
      nominator = (dfLocal['value'].cumsum()-dfLocal['initial_value'])
      denominator = (dfLocal['target_value']-dfLocal['initial_value'])
      metric = nominator/denominator*100
    else:
      nominator = (dfLocal['value']-dfLocal['initial_value'])
      denominator = (dfLocal['target_value']-dfLocal['initial_value'])
      metric = nominator/denominator*100

  elif option == 'performance':  
    if unit=='chf' or unit=='amount':
        denominator = (dfGlobal['value'].cumsum().max()-dfGlobal['value'].cumsum().min())
        if denominator == 0:
          metric = 0
        else:
          nominator = (dfLocal['value'].cumsum()-dfGlobal['value'].cumsum().min())
          metric = nominator/denominator*100
    else: 
        denominator = (dfGlobal['value'].max()-dfGlobal['value'].min())
        if denominator == 0:
          metric = 0
        else:
          nominator = (dfLocal['value']-dfGlobal['value'].min())
          metric = nominator/denominator*100
  return list(metric.round(0))

def breakString(input_string, max_length):
    if len(input_string) <= max_length:
        return input_string
    
    #Find the last occurrence of a space before the max_length
    breakIndex = input_string.rfind(' ', 0, max_length)
    
    if breakIndex == -1:
        return input_string  #Return the original string if no suitable break point is found
    
    # Return the string broken into two lines at the found space
    return input_string[:breakIndex] + '\n' + input_string[breakIndex + 1:]


def dictToJs(obj):
  """
  Convert a Python object to a JS string
  Args:
    obj: Python object
  Returns:
    JS string
  """
  if isinstance(obj, bool):
      return str(obj).lower()
  if isinstance(obj, str):
      return f'"{obj}"'
  if isinstance(obj, (int, float)):
      return str(obj)
  if isinstance(obj, list):
      return '[' + ', '.join(dictToJs(e) for e in obj) + ']'
  if isinstance(obj, dict):
      return '{' + ', '.join(f'{k}: {dictToJs(v)}' for k, v in obj.items()) + '}'

def launchHtml(codeSnippetJS, prefix, kpi ,year):
  """
  Launch the HTML file
  Args:
    codeSnippetJS: code snippet to be added to the HTML file
    prefix: prefix of the HTML file for file naming
    kpi: KPI name
    year: year
    
  Returns:
    None
  """
 
  # Creating the HTML file
  ROOT = "/Users/diana/Dropbox/_hackathon/deploy_2023/_visualisations/_echarts/"
  fileName = prefix+'_'+re.sub('[^A-Za-z0-9]+', '_', kpi)+ "_" + str(year) + ".html"
  pathFile = os.path.join(ROOT,fileName)
  file_html = open(pathFile, "w")

  # Adding the input data to the HTML file
  file_html.write('''<html>
    <head>
      <meta charset="utf-8" />
      <title>ECharts</title>
      <!-- Include the ECharts library from a CDN -->
      <script src="https://cdn.jsdelivr.net/npm/echarts@5/dist/echarts.min.js"></script>
    </head>
    <body>
    
      <!-- Prepare a DOM container with width and height -->
      <div id="main" style="width: 600px;height:400px;"></div>
            
      <!-- Include Google Fonts for Montserrat -->
      <link href="https://fonts.googleapis.com/css2?family=Montserrat:wght@400;700&display=swap" rel="stylesheet">
      
      <script type="text/javascript">
        '''
        +codeSnippetJS+
        '''
      </script>
    </body>
  </html>''')
  
  # Saving the data into the HTML file
  file_html.close()
  
  # 1st method how to open html files in chrome using
  filename = 'file:///' + os.path.abspath(pathFile)
  webbrowser.open_new_tab(filename)
  
def initials(phrase):
  words = phrase.split()
  result = ""
  for word in words:
      result += word[0].upper()
  return result

In [2]:
PATH_TO_FILE = '/Users/diana/Dropbox/_hackathon/deploy_2023/_data/pj_time_series_smooth_progress_performance_2014_2023.csv'
df = pd.read_csv(PATH_TO_FILE).round(0)  #take out decimals to prepare for plots

# Convert the 'date' column to datetime type
df.date = pd.to_datetime(df.date)

# remove dropped index if there is one
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns=['Unnamed: 0'])
    
df[['kpi','circle','periodicity','unit']].drop_duplicates()

,kpi,circle,periodicity,unit
0,additional monetization/savings from CRM,Digital,quarter,chf
40,private donations,Fundraising,month,chf
160,involuntary headcount change (FTE),HR,month,%
280,share of teams constituted as circles,HR,month,%_cumulative
292,share short term leave,HR,month,%
412,additional monetization/savings from programs,Program,quarter,chf
452,reachability,Programs - Children - Counceling,quarter,%
492,count leads,Programs - Parents -Online,month,amount
612,count sessions on .projuventute.ch,Programs - Parents -Online,month,amount
732,net promoter score,Programs - Parents -Online,year,score


In [3]:
# Creating subplots for the time series of each KPI 
kpis = df.kpi.unique().tolist()
fig = make_subplots(rows=5, cols=2, subplot_titles=kpis, shared_xaxes=False)

row = 1
col = 1
for kpi in kpis:
    dfKpi = df[df['kpi'] == kpi]
    trace = go.Scatter(x=dfKpi['date'], y=dfKpi['value'], name=kpi, mode='lines')
    fig.add_trace(trace, row=row, col=col)
    col += 1
    if col > 2:
        col = 1
        row += 1

fig.update_layout(height=1200, width=1000, title_text='Time Series of Each KPI')
fig.show()

# Create plots based on conditions

## Actual KPIs data plots

In [4]:
# inputs functions
circles = df.circle.unique().tolist()

monthList = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
              'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# palette used for actual data plots
palettePurple=[ '#4E17B0', #Base color
                '#833FFF', 
                '#CFB5FF'] #Lighter shade 
paletteMagenta = ['#D00071', #Base color                 
                '#FF79C1',
                '#FFD0E9']#lighter tone

                
# palette used for progress data plots
paletteYellow = [   '#e5a200', #Base color                 
                    '#FBBB21',
                    '#FECC33',
                    '#FEF9E9']#lighter tone
                    
# palette used for circles and kpis
paletteRed = [  '#D63503', #Base color                 
                '#FF8A65',
                '#FCEFEB']#lighter tone

paletteGreen = ['#1C6420', #Base color                 
                '#66BB6A',
                '#BFEAC0']#lighter tone

paletteCyan = ['#08C2DB', #Base color                 
                '#93E8F4',
                '#C5F7FD']#lighter tone

paletteBlue = ['#072490', #Base color                 
                '#6686FF',
                '#B2C3FF']#lighter tone

paletteGray = ['#656565', #Base color                 
                '#ABA8A3',
                '#F0EEEB']#lighter tone

paletteOlive = ['#5B6A00', #Base color
                '#B5D00D',
                '#E8F49A']

paletteTerracota = ['#8C5009', #Base color
                    '#F59019',
                    '#FFCB8D']

In [5]:
def plotActualDataJS(df,
                     circle,
                     year,
                     month,
                     palette=[palettePurple,paletteMagenta],
                     monthList=monthList):
    
    # make selection from year start until selected month
    dfTemp = df.loc[
        (df.circle == circle) &
        (df.date <= pd.to_datetime(str(year)+'-'+str(month)+'-01'))&
        (df.date >= pd.to_datetime(str(year)+'-01-01'))
        ]
    
    # check for unit chf and periodicity month  >> line plot together
    if  dfTemp.empty == False:

        # find selected kpis
        kpis = dfTemp.kpi.unique().tolist()
        
        for i,k in enumerate(kpis):
        
            # detect baseline, target, periodicity and unit
            baseline = dfTemp.loc[dfTemp.kpi == k,'initial_value'].tolist()[-1]
            target = dfTemp.loc[dfTemp.kpi == k,'target_value'].tolist()[-1]
            periodicity = dfTemp.loc[dfTemp.kpi == k,'periodicity'].unique().tolist()[0]
            unit = dfTemp.loc[dfTemp.kpi == k,'unit'].unique().tolist()[0]
            
            # define line plots for line plots (chf and monthly KPIS)          
            if periodicity == 'month':
                dfTempLocal = dfTemp.loc[(df.kpi == k),['date','value']]
                # define months for monthly graphic
                months = dfTempLocal.date.dt.month.unique()
                monthsRange = monthList[months[0]-1:months[-1]]
                
                if  unit == 'chf'or unit == 'amount':
                    # Create 'aggregated' that cumulatively sums the 'value' column
                    values = dfTempLocal.value.to_list()
                    valuesCum = dfTempLocal['value'].cumsum().to_list()
                    
                    # define max and min value for y axis
                    maxValueY = max(values+[baseline]+[target]+valuesCum)
                    minValueY = min(values+[baseline]+[target])
                    
                    varData = """
                    // Initialize the echarts instance based on the prepared dom
                    var myChart = echarts.init(document.getElementById('main'));

                    // Apply Montserrat font to ECharts container
                    document.getElementById('main').style.fontFamily ='Montserrat, sans-serif';

                    // Prepare data
                    var data = [
                        {
                        name: '"""+k.title()+"""',
                        type: 'line',
                        data: """+str(values)+""",
                        itemStyle: {color: '"""+palette[0][0]+"""'},
                        markLine: {
                            silent: true,
                            data: [
                                {
                                    yAxis: """+str(target)+""",
                                    lineStyle: {color: '#D00071', width: 1.5, type: 'dotted'},
                                    label: {
                                        show: true,
                                        position: 'end',
                                        formatter: 'Target',
                                        color: '#000000'
                                    }
                                },
                                {
                                    yAxis: """+str(baseline)+""",
                                    lineStyle: {color: '#D00071', width: 1.5, type: 'dotted'},
                                    label: {
                                        show: true,
                                        position: 'end',
                                        formatter: 'Baseline',
                                        color: '#000000'
                                    }
                                }
                            ],
                            symbol: 'none'
                        }
                    },
                    {
                    name: 'Cumulative',
                        type: 'line',
                        data: """+str(valuesCum)+""", // Add cumulative data here
                            areaStyle: {
                                color: {
                                    type: 'linear',
                                    x: 0,
                                    y: 0,
                                    x2: 0,
                                    y2: 1,
                                    colorStops: [{
                                        offset: 0, color: 'rgba(208, 0, 113, 0.5)' // color at 0% position
                                    }, {
                                        offset: 1, color: 'rgba(208, 0, 113, 0)' // color at 100% position
                                    }],
                                    global: false // false by default
                                }
                            },  
                        itemStyle: {color: '"""+palette[1][0]+"""'}  
                    }
                    ];
                    """
                elif  unit == '%_cumulative':
                    # Create the 'aggregated' column that cumulatively sums the 'value' column
                    values = dfTempLocal.value.to_list()
                    
                    # define max and min value for y axis
                    maxValueY = max(values+[baseline]+[target])
                    minValueY = min(values+[baseline]+[target])
                    
                    varData = """
                    // Initialize the echarts instance based on the prepared dom
                    var myChart = echarts.init(document.getElementById('main'));

                    // Apply Montserrat font to ECharts container
                    document.getElementById('main').style.fontFamily ='Montserrat, sans-serif';
                    
                    // Prepare data
                    var data = [
                        {
                        name: 'Cumulative """+k.title()+"""',
                            type: 'line',
                            data: """+str(values)+""", // Add cumulative data here
                                areaStyle: {
                                    color: {
                                        type: 'linear',
                                        x: 0,
                                        y: 0,
                                        x2: 0,
                                        y2: 1,
                                        colorStops: [{
                                            offset: 0, color: 'rgba(208, 0, 113, 0.5)' // color at 0% position
                                        }, {
                                            offset: 1, color: 'rgba(208, 0, 113, 0)' // color at 100% position
                                        }],
                                        global: false // false by default
                                    }
                                },  
                            itemStyle: {color: '"""+palette[1][0]+"""'},
                            markLine: {
                                silent: true,
                                data: [
                                    {
                                        yAxis: """+str(target)+""",
                                        lineStyle: {color: '#D00071', width: 1.5, type: 'dotted'},
                                        label: {
                                            show: true,
                                            position: 'end',
                                            formatter: 'Target',
                                            color: '#000000'
                                        }
                                    },
                                    {
                                        yAxis: """+str(baseline)+""",
                                        lineStyle: {color: '#D00071', width: 1.5, type: 'dotted'},
                                        label: {
                                            show: true,
                                            position: 'end',
                                            formatter: 'Baseline',
                                            color: '#000000'
                                        }
                                    }
                                ],
                                symbol: 'none'
                            }
                        }
                    ];
                    """
                    
                else: 
                    #includes: %, score and %_cumulative
                    values = dfTempLocal.value.to_list()
                    # define max and min value for y axis
                    maxValueY = max(values+[baseline]+[target])
                    minValueY = min(values+[baseline]+[target])
                    
                    varData = """
                    // Initialize the echarts instance based on the prepared dom
                    var myChart = echarts.init(document.getElementById('main'));

                    // Apply Montserrat font to ECharts container
                    document.getElementById('main').style.fontFamily ='Montserrat, sans-serif';

                    // Prepare data
                    var data = [
                        {
                        name: '"""+k.title()+"""',
                        type: 'line',
                        data: """+str(values)+""",
                        itemStyle: {color: '"""+palette[0][0]+"""'},
                        markLine: {
                            silent: true,
                            data: [
                                {
                                    yAxis: """+str(np.mean(values))+""",
                                    lineStyle: {color: '#D00071', width: 1.5, type: 'dotted'},
                                    label: {
                                        show: true,
                                        position: 'end',
                                        formatter: 'Mean',
                                        color: '#000000'
                                    }
                                },
                                {
                                    yAxis: """+str(target)+""",
                                    lineStyle: {color: '#D00071', width: 1.5, type: 'dotted'},
                                    label: {
                                        show: true,
                                        position: 'end',
                                        formatter: 'Target',
                                        color: '#000000'
                                    }
                                },
                                {
                                    yAxis: """+str(baseline)+""",
                                    lineStyle: {color: '#D00071', width: 1.5, type: 'dotted'},
                                    label: {
                                        show: true,
                                        position: 'end',
                                        formatter: 'Baseline',
                                        color: '#000000'
                                    }
                                }
                            ],
                            symbol: 'none'
                        }
                    }
                    ];
                    """
                
                varOptions = """
                // Configure chart options
                var option = {
                    title: {
                        text: '"""+k.title()+"""',
                        textStyle: {
                            fontFamily: 'Montserrat',
                            fontSize: 12
                        }
                    },
                    tooltip: {
                        trigger: 'axis',
                        formatter: function (params) {
                            return params[0].axisValueLabel + '<br/>' +
                                params.map(function (item) {
                                    return item.marker + ' ' + item.seriesName + ': ' + item.data;
                                }).join('<br/>');
                        },
                        textStyle: {
                            fontFamily: 'Montserrat',
                            fontSize: 12
                        }
                    },
                    legend: {
                        data: ['"""+k+"""'], //KPIs
                        bottom: 0,
                        textStyle: {
                            fontFamily: 'Montserrat'
                        }
                    },
                    xAxis: {
                        type: 'category',
                        boundaryGap: true,
                        data: """+str(monthsRange)+""", //MONTHS
                        axisLabel: {
                            textStyle: {
                                fontFamily: 'Montserrat'
                            }
                        }
                    },
                    yAxis: {
                        type: 'value',
                        scale: true,  
                        name: '""" + unit + """',  // Add this line to label the y-axis
                        max : """+str(maxValueY)+""",
                        min : """+str(minValueY)+""",
                        //nameLocation: 'center',
                        //nameGap: 50,
                        nameTextStyle: {   // Optional: Style the y-axis label
                            color: '#000',
                            fontFamily: 'Montserrat',
                            fontSize: 14
                        },
                        axisLabel: {
                            textStyle: {
                                fontFamily: 'Montserrat'
                            }
                        }
                    },
                    series: data
                };

                // Display the chart using the configuration items and data just specified.
                myChart.setOption(option);
                """
                codeSnippetJS = varData + varOptions
                
            # define bar plots, stacked or not (quarter and/or % KPIs)
            elif periodicity == 'quarter':
                dfTempLocal = dfTemp.loc[(df.kpi == k)&
                    (df.date.dt.month.isin([3,6,9,12])),
                    ['date','value']]
                    
                # define months for quarters
                months = dfTempLocal.date.dt.month.unique()
                monthsRange = [monthList[i-1] for i in months]
                    
                if  unit == 'chf' or unit == 'amount':
                    # Create 'aggregated' values that cumulatively sums the 'value' column
                    values = dfTempLocal.value.to_list()
                    valuesCum = dfTempLocal['value'].cumsum().to_list()
                    
                    # define max and min value for y axis
                    maxValueY = max(values+[baseline]+[target]+valuesCum)
                    minValueY = min(values+[baseline]+[target])
                    
                    varData = """
                    // Initialize the echarts instance based on the prepared dom
                    var myChart = echarts.init(document.getElementById('main'));

                    // Apply Montserrat font to ECharts container
                    document.getElementById('main').style.fontFamily ='Montserrat, sans-serif';

                    // Prepare data
                    var data = [
                        {
                        name: 'Cumulative',
                            type: 'line',
                            data: """+str(valuesCum)+""", // Add cumulative data here
                                areaStyle: {
                                    color: {
                                        type: 'linear',
                                        x: 0,
                                        y: 0,
                                        x2: 0,
                                        y2: 1,
                                        colorStops: [{
                                            offset: 0, color: 'rgba(208, 0, 113, 0.5)' // color at 0% position
                                        }, {
                                            offset: 1, color: 'rgba(208, 0, 113, 0)' // color at 100% position
                                        }],
                                        global: false // false by default
                                    }
                                },  
                            itemStyle: {color: '"""+palette[1][0]+"""'}  
                        },
                        {
                        name: '"""+k.title()+"""',
                        type: 'bar', stack: 'total',
                        data: """+str(values)+""",
                        itemStyle: {color: '"""+palette[0][0]+"""'},
                        markLine: {
                            silent: true,
                            data: [
                                {
                                    yAxis: """+str(target)+""",
                                    lineStyle: {color: '#D00071', width: 1.5, type: 'dotted'},
                                    label: {
                                        show: true,
                                        position: 'end',
                                        formatter: 'Target',
                                        color: '#000000'
                                    }
                                },
                                {
                                    yAxis: """+str(baseline)+""",
                                    lineStyle: {color: '#D00071', width: 1.5, type: 'dotted'},
                                    label: {
                                        show: true,
                                        position: 'end',
                                        formatter: 'Baseline',
                                        color: '#000000'
                                    }
                                }
                            ],
                            symbol: 'none'
                        }
                    },

                    ];
                    """
                    
                elif unit == '%_cumulative':
                    # values aggregated by default
                    values = dfTempLocal.value.to_list()
                    
                    # define max and min value for y axis
                    maxValueY = max(values+[baseline]+[target])
                    minValueY = min(values+[baseline]+[target])
                    
                    varData = """
                    // Initialize the echarts instance based on the prepared dom
                    var myChart = echarts.init(document.getElementById('main'));

                    // Apply Montserrat font to ECharts container
                    document.getElementById('main').style.fontFamily ='Montserrat, sans-serif';

                    // Prepare data
                    var data = [
                        {
                        name: '"""+k.title()+"""',
                        type: 'bar', 
                        data: """+str(values)+""",
                        itemStyle: {color: '"""+palette[0][0]+"""'},
                        markLine: {
                            silent: true,
                            data: [
                                {
                                    yAxis: """+str(target)+""",
                                    lineStyle: {color: '#D00071', width: 1.5, type: 'dotted'},
                                    label: {
                                        show: true,
                                        position: 'end',
                                        formatter: 'Target',
                                        color: '#000000'
                                    }
                                },
                                {
                                    yAxis: """+str(baseline)+""",
                                    lineStyle: {color: '#D00071', width: 1.5, type: 'dotted'},
                                    label: {
                                        show: true,
                                        position: 'end',
                                        formatter: 'Baseline',
                                        color: '#000000'
                                    }
                                }
                            ],
                            symbol: 'none'
                        }
                    },
                    {
                    name: 'Cumulative',
                        type: 'line',
                        data: """+str(values)+""", // Add cumulative data here
                            areaStyle: {
                                color: {
                                    type: 'linear',
                                    x: 0,
                                    y: 0,
                                    x2: 0,
                                    y2: 1,
                                    colorStops: [{
                                        offset: 0, color: 'rgba(208, 0, 113, 0.5)' // color at 0% position
                                    }, {
                                        offset: 1, color: 'rgba(208, 0, 113, 0)' // color at 100% position
                                    }],
                                    global: false // false by default
                                }
                            },  
                        itemStyle: {color: '"""+palette[1][0]+"""'}  
                    }
                    ];
                    """
                else:
                    # applies to % and score. No aggregation
                    values = dfTempLocal.value.to_list()
                    
                    # define max and min value for y axis
                    maxValueY = max(values+[baseline]+[target])
                    minValueY = min(values+[baseline]+[target])
                
                    varData = """
                    // Initialize the echarts instance based on the prepared dom
                    var myChart = echarts.init(document.getElementById('main'));

                    // Apply Montserrat font to ECharts container
                    document.getElementById('main').style.fontFamily ='Montserrat, sans-serif';

                    // Prepare data
                    var data = [
                        {
                        name: '"""+k.title()+"""',
                        type: 'bar', stack: 'total',
                        data: """+str(values)+""",
                        itemStyle: {color: '"""+palette[0][0]+"""'},
                        markLine: {
                            silent: true,
                            data: [
                                {
                                    yAxis: """+str(np.mean(values))+""",
                                    lineStyle: {color: '#D00071', width: 1.5, type: 'dotted'},
                                    label: {
                                        show: true,
                                        position: 'end',
                                        formatter: 'Mean',
                                        color: '#000000'
                                    }
                                },
                                {
                                    yAxis: """+str(target)+""",
                                    lineStyle: {color: '#D00071', width: 1.5, type: 'dotted'},
                                    label: {
                                        show: true,
                                        position: 'end',
                                        formatter: 'Target',
                                        color: '#000000'
                                    }
                                },
                                {
                                    yAxis: """+str(baseline)+""",
                                    lineStyle: {color: '#D00071', width: 1.5, type: 'dotted'},
                                    label: {
                                        show: true,
                                        position: 'end',
                                        formatter: 'Baseline',
                                        color: '#000000'
                                    }
                                }
                            ],
                            symbol: 'none'
                        }
                    }
                    ];
                    """
                
                varOptions = """
                // Configure chart options
                var option = {
                    title: {
                        text: '"""+k.title()+"""',
                        textStyle: {
                            fontFamily: 'Montserrat',
                            fontSize: 12
                        }
                    },
                    tooltip: {
                        trigger: 'axis',
                        formatter: function (params) {
                            return params[0].axisValueLabel + '<br/>' +
                                params.map(function (item) {
                                    return item.marker + ' ' + item.seriesName + ': ' + item.data;
                                }).join('<br/>');
                        },
                        textStyle: {
                            fontFamily: 'Montserrat',
                            fontSize: 12
                        }
                    },
                    legend: {
                        data: ['"""+k+"""'], //KPIs
                        bottom: 0,
                        textStyle: {
                            fontFamily: 'Montserrat'
                        }
                    },
                    xAxis: {
                        type: 'category',
                        boundaryGap: true,
                        data: """+str(monthsRange)+""", //MONTHS
                        axisLabel: {
                            textStyle: {
                                fontFamily: 'Montserrat'
                            }
                        }
                    },
                    yAxis: {
                        type: 'value',
                        scale: true,  
                        name: '""" + unit + """',  // Add this line to label the y-axis
                        max : """+str(maxValueY)+""",
                        min : """+str(minValueY)+""",
                        //nameLocation: 'center',
                        //nameGap: 50,
                        nameTextStyle: {   // Optional: Style the y-axis label
                            color: '#000',
                            fontFamily: 'Montserrat',
                            fontSize: 14
                        },
                        axisLabel: {
                            textStyle: {
                                fontFamily: 'Montserrat'
                            }
                        }
                    },
                    series: data
                };

                // Display the chart using the configuration items and data just specified.
                myChart.setOption(option);
                """
                codeSnippetJS = varData + varOptions
                
            # define gauge plots for GAUGES (yearly KPIs)  
            elif periodicity == 'year':
                dfTempLocal = dfTemp.loc[(df.kpi == k)&
                                    (df.date.dt.month.isin([12])),
                                    ['date','value']]
                if dfTempLocal.empty==True:
                    dfTempLocal = dfTemp.loc[(df.kpi == k)&
                    (df.date.dt.month.isin([month])),
                    ['date','value']]
                    
                # define months for quarters
                months = dfTempLocal.date.dt.month.unique()
                monthsRange = [monthList[i-1] for i in months]
                    
                values = dfTempLocal.value.mean()
                
                codeSnippetJS = """
                // Initialize the echarts instance based on the prepared dom
                var myChart = echarts.init(document.getElementById('main'));

                // Apply Montserrat font to ECharts container
                document.getElementById('main').style.fontFamily = 'Montserrat, sans-serif';

                // Prepare data
                var value ="""+str(values)+""";  // This is the value to be displayed on the gauge

                // Configure chart options
                var option = {
                    title: {
                        text: '"""+k.title()+"""',
                        textStyle: {
                            fontFamily: 'Montserrat',
                            fontSize: 12
                        }
                    },
                    tooltip: {
                        formatter: '{a} <br/>{b} : {c}%'
                    },
                    series: [
                        {
                            name: 'Net Promoter Score', //Kpi Name here
                            type: 'gauge',
                            center: ['50%', '60%'],
                            startAngle: 180,
                            endAngle: 0,
                            min: """+str(baseline)+""",
                            max: """+str(target)+""",
                            splitNumber: 10,
                            itemStyle: {
                                color: (value < -33) ? '#D63503' : (value < 33) ? '#FBBB21' : '#1C6420'  // Conditional color based on value
                            },
                            progress: {
                                show: true,
                                width: 40
                            },
                            pointer: {
                                show: false
                            },
                            axisLine: {
                                lineStyle: {
                                    width: 40,
                                    color: [[1, '#E0E0E0']]  // Background color set to light gray
                                }
                            },
                            axisTick: {
                                distance: -55,
                                splitNumber: 10,
                                lineStyle: {
                                    width: 1,
                                    color: '#999'
                                }
                            },
                            splitLine: {
                                distance: -60,
                                length: 14,
                                lineStyle: {
                                    width: 2,
                                    color: '#999'
                                }
                            },
                            axisLabel: {
                                distance: 5,
                                color: '#999',
                                fontSize: 10,
                                fontFamily: 'Montserrat'
                            },
                            anchor: {
                                show: false
                            },
                            title: {
                                show: true
                            },
                            detail: {
                                valueAnimation: true,
                                width: '60%',
                                lineHeight: 40,
                                borderRadius: 8,
                                offsetCenter: [0, '-15%'],
                                fontSize: 30,
                                fontWeight: 'bolder',
                                fontFamily: 'Montserrat',
                                formatter: '{value} """+initials(k)+"""', //Kpi Name here
                                color: 'inherit'
                            },
                            data: [
                                {
                                    value: value,
                                    itemStyle: {
                                        color: (value < -33) ? '#D63503' : (value < 33) ? '#FBBB21' : '#1C6420'  // Conditional color based on value
                                    }
                                }
                            ]
                        }
                    ]
                };

                // Display the chart using the configuration items and data just specified.
                myChart.setOption(option);
                """
            launchHtml(codeSnippetJS, 'kpi', k ,year)

    else:
        print('there is no values with this combination')
                

In [22]:
def plotMetricsJS(df,
                  circle,
                  year,
                  month,
                  option = 'progress', #'performance' >>> what should be displayed
                  plot = 'bar',#'circular','donut','radial'
                  paletteProgress = paletteYellow,
                  palettes=[paletteOlive,paletteGreen,paletteCyan,
                            paletteBlue,paletteRed,paletteTerracota],
                  monthList=monthList):
    
    # make selection from year start until selected month
    dfTemp = df.loc[
        (df.circle == circle) &
        (df.date <= pd.to_datetime(str(year)+'-'+str(month)+'-01'))&
        (df.date >= pd.to_datetime(str(year)+'-01-01'))
        ]
    dfPerformance = df.loc[
        (df.circle == circle) &
        (df.date <= pd.to_datetime(str(year)+'-'+str(month)+'-01'))
        ]
    
    if  dfTemp.empty == False:
        # find selected kpis
        kpis = dfTemp.kpi.unique().tolist()

        if plot == 'bar':
            kpiMetrics = []
            for i,k in enumerate(kpis):
                periodicity = dfTemp.loc[(df.kpi == k),'periodicity'].unique()[0]
                # define time axis global
                monthsRange = [monthList[i] for i in range(month)]
                
                if periodicity == 'month':
                    dfTempLocal = dfTemp.loc[(df.kpi == k)]
                    # calculate on the fly, rather than from df
                    values = calculateMetric(dfTempLocal,
                                             dfPerformance.loc[(dfPerformance.kpi == k)],
                                             option)
          
                elif periodicity == 'quarter':
                    # for now only works with circles that are only quarterly
                    dfTempLocal = dfTemp.loc[(df.kpi == k)&
                                        (df.date.dt.month.isin([3,6,9,12]))]
                    
                    # calculate on the fly, rather than from df
                    values = calculateMetric(dfTempLocal,
                                             dfPerformance.loc[(dfPerformance.kpi == k)],
                                             option)
              
                    # define time axis locally
                    months = dfTempLocal.date.dt.month.unique().tolist()
                    monthsRange = [monthList[i-1] for i in months]
                    
                    # use in the future if needed to adapt to yearly plots
                    # values = [[v]*3 for v in values]
                    # values = [item for sublist in values for item in sublist]
                
                elif periodicity == 'year':
                    dfTempLocal = dfTemp.loc[(df.kpi == k)&
                                (df.date.dt.month.isin([12]))]
                    if dfTempLocal.empty==True:
                        dfTempLocal = dfTemp.loc[(df.kpi == k)&
                        (df.date.dt.month.isin([month]))]
     
                    # calculate on the fly, rather than from df
                    values = calculateMetric(dfTempLocal,
                                             dfPerformance.loc[(dfPerformance.kpi == k)],
                                             option)
              
                    values = [values]*month
                    values = [item for sublist in values for item in sublist]

                kpiMetric = {
                                "name": k.title(),
                                "type": "bar",
                                "stack": "total",
                                "itemStyle": {"color": paletteProgress[i]},
                                "label": {"show": True},
                                "emphasis": {"focus": "series"},
                                "data": values,
                            }        

                # Convert the dictionary to a JSON string with a custom serializer
                kpiMetric = dictToJs(kpiMetric)
                kpiMetrics.append(kpiMetric)
                
            kpiMetricsString = ",".join(kpiMetrics)
            
            codeSnippetJS = """
            // Initialize the echarts instance based on the prepared dom
            var myChart = echarts.init(document.getElementById('main'));

            // Apply Montserrat font to ECharts container
            document.getElementById('main').style.fontFamily = 'Montserrat, sans-serif';
            
            var option = {
                title: {
                    text: 'Stacked """+option.title()+""" (%) Per KPI Over Time',  // Add this line to set the title
                    left: '0%',  // Optional: Align the title
                    top: 'top',  // Optional: Set the position of the title
                    textStyle: {  // Optional: Style the title
                        color: '#000',
                        fontFamily: 'Montserrat',
                        fontSize: 12
                    }                
                },
                tooltip: {
                    trigger: 'axis',
                    axisPointer: {
                    // Use axis to trigger tooltip
                    type: 'shadow' // 'shadow' as default; can also be 'line' or 'shadow'
                    },
                    textStyle: { 
                        fontFamily: 'Montserrat'
                    }
                },
                legend: {
                    top: 30, //use this to move legend below title
                    itemGap: 30, // space between legend items
                    textStyle: {  // Added textStyle attribute here to set the font for legend text
                        fontFamily: 'Montserrat',
                        fontSize: 10   
                    }
                },
                grid: {
                    left: '3%',
                    right: '4%',
                    bottom: '3%',
                    top: '30%',  // Increased the space between the grid and the title
                    containLabel: true
                },
                xAxis: {
                    type: 'category',
                    data: """+str(monthsRange)+""",
                    axisLabel: {  // Added axisLabel attribute here to set the font for axis labels
                        fontFamily: 'Montserrat'
                    }
                },
                yAxis: {
                    type: 'value',
                    name: '"""+option.title()+"""',
                    nameLocation: 'center',
                    nameGap: 50,
                    nameTextStyle: {
                        color: '#000',
                        fontSize: 14,
                        fontFamily: 'Montserrat'  // Added fontFamily here
                    },
                    axisLabel: {  // Added axisLabel attribute here to set the font for axis labels
                        fontFamily: 'Montserrat'
                    }
                },
                series: [ """+kpiMetricsString+"""
                ]
            };
            
            var myChart = echarts.init(document.getElementById('main'));
            myChart.setOption(option);
            """
        elif plot == 'circular':
            #radial bars, represent cumulative progress on separate kpis
            kpiMetrics = []
            for i,k in enumerate(kpis):
                #take the last month value in the range because cumulative
                # calculate on the fly, rather than from df column
                dfTempLocal = dfTemp.loc[(dfTemp.kpi == k)]
                value = calculateMetric(dfTempLocal,
                                         dfPerformance.loc[(dfPerformance.kpi == k)],
                                         option)[-1]
                
                kpiMetric = {
                    "value": value+0.0001, # add 0.0001 to avoid 0.0, otherwise there will be a bug with the fonts
                    "itemStyle": {"color": paletteProgress[i]}
                            }        

                # Convert the dictionary to a JSON string with a custom serializer
                kpiMetric = dictToJs(kpiMetric)
                kpiMetrics.append(kpiMetric)
                
            kpiMetricsString = ",".join(kpiMetrics)
            
            codeSnippetJS = """
            // Initialize the echarts instance based on the prepared dom
            var myChart = echarts.init(document.getElementById('main'));

            // Apply Montserrat font to ECharts container
            document.getElementById('main').style.fontFamily = 'Montserrat, sans-serif';
            
            var option = {
                title: [
                    {
                        text: 'Average """+option.title()+""" (%) Per KPI',
                        left: '0%',  // Optional: Align the title
                        top: 'top',  // Optional: Set the position of the title
                        textStyle: {
                            color: '#000',
                            fontFamily: 'Montserrat',
                            fontSize: 12
                        },
                        left: '0%'
                    }
                ],
                series: {
                    name: 'Progress',
                    type: 'bar',
                    data: ["""+kpiMetricsString+"""
                    ],
                    coordinateSystem: 'polar',
                    label: {
                        show: false,
                        position: 'middle',
                        formatter: '{b}: {c}',
                        textStyle: {
                            fontFamily: 'Montserrat'
                        }
                    }
                },
                tooltip: { // formated here to show round values at display
                    trigger: 'axis',
                    axisPointer: {
                        type: 'shadow'
                    },
                    textStyle: { 
                        fontFamily: 'Montserrat'
                    },
                    formatter: function(params) {
                        var tip = params[0].axisValueLabel + '<br>';
                        params.forEach(function(param) {
                            tip += param.marker + ' ' + param.seriesName + ': ' + Math.round(param.value) + '<br>';
                        });
                        return tip;
                    }
                },
                
                polar: {
                    radius: [30, '80%']
                },
                angleAxis: {
                    max: 100,
                    startAngle: 90,
                    axisLabel: {
                        textStyle: {
                            fontFamily: 'Montserrat'
                        }
                    }
                },
                radiusAxis: {
                    type: 'category',
                    data: """+str(kpis)+""", // introduce kpi names for circle
                    show: false, // hide labels in front of bars, too messy
                    axisLabel: {
                        textStyle: {
                            fontFamily: 'Montserrat'
                        }
                    }
                }
            };

            var myChart = echarts.init(document.getElementById('main'));
            myChart.setOption(option);    
            """
            
        elif plot == 'donut':
            # the donut shows an average progress for the whole circle
            #take the last month value in the range since its cumulative
            values = [calculateMetric(dfTemp.loc[dfTemp.kpi == k],
                                        dfPerformance.loc[(dfPerformance.kpi == k)],
                                        option)[-1] for k in kpis]
            value = np.mean(values)
                
            #baseline = 0
            target = 100
            
            codeSnippetJS = """
            // Initialize the echarts instance based on the prepared dom
            var myChart = echarts.init(document.getElementById('main'));

            // Apply Montserrat font to ECharts container
            document.getElementById('main').style.fontFamily = 'Montserrat, sans-serif';
    
            var value = """+str(value)+""".toString();  // This is the value you want to display

            var option = {
                title: [
                    {
                        text: 'Average """+option.title()+""" (%) For Entire Circle, All KPIs Included',
                        left: '0%',  
                        top: 'top',  
                        textStyle: {
                            color: '#000',
                            fontFamily: 'Montserrat',
                            fontSize: 12
                        }
                    }
                ],
                tooltip: {
                    trigger: 'item',
                    textStyle: { 
                        fontFamily: 'Montserrat'
                    },
                    formatter: function(params) {
                        return params.name + ': ' + params.value.toFixed(1) + '%';
                    }
                },
                series: [
                    {
                        name: '"""+option.title()+"""',
                        type: 'pie',
                        radius: ['50%', '80%'],
                        data: [
                            {value: """+str(value)+""", name: '"""+option.title()+"""', itemStyle: {color: '#E5A200'}},
                            // This data entry will represent the unfilled portion of the donut
                            {value: """""+str(target-value)+""", name: 'Remaining', itemStyle: {color: '#F0EEEB'}}
                        ],
                        label: {
                            show: true,
                            position: 'center',
                            formatter: function(params) {
                                return params.value.toFixed(1) + '%';  // Updated to round the values displayed in the center
                            },
                            fontSize: 30,
                            fontWeight: 'bolder',
                            fontFamily: 'Montserrat',
                            color: 'inherit'
                        }
                    }
                ]
            };            
            myChart.setOption(option);   
            """
    
        elif plot == 'radial':
            # select metric from year start until selected month, all circles
            dfTempLocal = df.loc[
                (df.date <= pd.to_datetime(str(year)+'-'+str(month)+'-01'))&
                (df.date >= pd.to_datetime(str(year)+'-01-01'))]
            circles = dfTempLocal.circle.unique().tolist()
            
            kpisAll = []
            kpiMetricsAll = []
            for i,c in enumerate(circles):
                kpisCircle = dfTempLocal.loc[dfTempLocal.circle == c].kpi.unique().tolist()
                # kpisAll.append(kpisCircle)
                kpiMetrics = []
                for j,k in enumerate(kpisCircle):
                    value = calculateMetric(dfTempLocal.loc[(dfTempLocal.kpi == k)],
                                            dfPerformance.loc[(dfPerformance.kpi == k)],
                                            option)[-1]+ 0.0001
                    kpiMetric = {"value": value, "name": k, "itemStyle": {"color": palettes[i][j]}}
                    kpisAll.append("C"+str(i+1)+": "+k)
                    
                    # Convert the dictionary to a JSON string with a custom serializer
                    kpiMetric = dictToJs(kpiMetric)
                    kpiMetrics.append(kpiMetric)
                kpiMetricsAll.append(kpiMetrics)
            
            kpisAll = [breakString(s.replace('/', ' / '), 30) for s in kpisAll]
            kpiMetricsAll = [item for sublist in kpiMetricsAll for item in sublist]
            kpiMetricsString = ",".join(kpiMetricsAll)
            
            codeSnippetJS = """
            // Initialize the echarts instance based on the prepared dom
            var myChart = echarts.init(document.getElementById('main'));

            // Apply Montserrat font to ECharts container
            document.getElementById('main').style.fontFamily = 'Montserrat, sans-serif';

            // Configure chart options
            option = {
                textStyle: {
                    fontFamily: 'Montserrat'
                },
                title: [
                    {
                        text: 'Collective """+option.title()+""" (%), All Circles And KPIs Included',
                        left: '0%',  // Adjusted: Align the title to the left
                        top: 'top',
                        textStyle: {
                            color: '#000',
                            fontFamily: 'Montserrat',  // Adjusted: Set the font to Montserrat
                            fontSize: 12  // Adjusted: Set the font size
                        }
                    }
                ],
                legend: {
                    show: true,
                    top: 35,  // Adjust this value based on the actual height of your title
                    type: 'scroll',
                    padding: [5, 10],
                    orient: 'horizontal',  // Change the orientation to horizontal
                    left: 'center',        // Center the legend
                    textStyle: {
                        fontSize: 10  // Adjusted the legend font size here
                    },
                },
                tooltip: {
                    trigger: 'item',
                    textStyle: { 
                        fontFamily: 'Montserrat'
                    },
                    formatter: function(params) {
                        return params.name + '<br>' + params.marker + ' ' + Math.round(params.value);
                    }
                },
                toolbox: {
                    show: false,
                    feature: {
                    mark: { show: false },
                    dataView: { show: true, readOnly: false },
                    restore: { show: true },
                    saveAsImage: { show: true }
                    }
                },
                series: [
                    {
                    name: 'KPIs Progress',
                    type: 'pie',
                    radius: [20, 150],
                    center: ['50%', '60%'],
                    roseType: 'area',
                    label: {
                        position: 'outside',  // Place labels outside the pie chart
                        formatter: '{b}: {d}%',  // Display the name and percentage
                        color: '#000', // Set label text color
                        fontSize: 0 //hide for now
                    },
                    labelLine: {
                        show: false,  // Display the label line
                        length: 15,  // Length of the line from the pie to the label
                        length2: 10  // Length from the end of the first line to the label
                    },
                    itemStyle: {
                        borderRadius: 0
                    },
                    data: ["""+kpiMetricsString+"""],
                    },

                ]
            };
            myChart.setOption(option);
            """
   
        launchHtml(codeSnippetJS,option+"_"+plot, circle ,year)
        
        
    else:
        print('there is no values with this combination')

In [17]:
def plotMetricsGatekeeperJS(df,
                  year,
                  month,
                  circle = 'all circles',
                  option = 'progress', #'performance' >>> what should be displayed
                  plot = 'bar',#'circular','donut','radial','heatmap','streamgraph'
                  palettes=[paletteOlive,paletteGreen,paletteCyan,
                            paletteBlue,paletteRed,paletteTerracota],
                  monthList=monthList):
    
    # make selection from year start until selected month
    dfTemp = df.loc[
        (df.date <= pd.to_datetime(str(year)+'-'+str(month)+'-01'))&
        (df.date >= pd.to_datetime(str(year)+'-01-01'))]
    dfPerformance = df.loc[(df.date <= pd.to_datetime(str(year)+'-'+str(month)+'-01'))]
    
    if  dfTemp.empty == False:
        # find selected kpis
        circles = dfTemp.circle.unique().tolist()
        # define time axis
        months = dfTemp.date.dt.month.sort_values().unique()
        monthsRange = [monthList[i-1] for i in months]

        if plot == 'bar':
            circleMetrics = []
            for i,c in enumerate(circles):
                kpis = dfTemp[dfTemp.circle == c].kpi.unique().tolist()
                collectValues = []
                for k in kpis:
                    periodicity = dfTemp.loc[(dfTemp.circle == c)
                                             &(df.kpi == k),'periodicity'].unique().tolist()[0]
                    if periodicity == 'month':
                        dfTempLocal = dfTemp.loc[(df.kpi == k)]
                        values = calculateMetric(dfTempLocal,
                                                 dfPerformance.loc[dfPerformance.kpi == k],
                                                 option)
                        collectValues.append(values)
                    
                    elif periodicity == 'quarter':
                        # for now only works with circles that are only quarterly
                        dfTempLocal = dfTemp.loc[(df.kpi == k)&
                                            (df.date.dt.month.isin([3,6,9,12]))]
                        
                        values = calculateMetric(dfTempLocal,
                                                 dfPerformance.loc[dfPerformance.kpi == k],
                                                 option) 
                                               
                        # use if needed to adapt to monthly plots
                        values = [[v]*3 for v in values]
                        values = [item for sublist in values for item in sublist]
                        collectValues.append(values)
                    
                    elif periodicity == 'year':
                        dfTempLocal = dfTemp.loc[(df.kpi == k)&
                                    (df.date.dt.month.isin([12]))]
                        if dfTempLocal.empty==True:
                            dfTempLocal = dfTemp.loc[(df.kpi == k)&
                            (df.date.dt.month.isin([month]))]

                        values = calculateMetric(dfTempLocal,
                                                 dfPerformance.loc[dfPerformance.kpi == k],
                                                 option)
                        values = [values]*month
                        values = [item for sublist in values for item in sublist]
                        collectValues.append(values)
                
                # combine all kpis in a mean value per circle        
                valuesMean = list(np.mean(np.transpose(collectValues),axis=1).round(2))

                circleMetric = {
                                "name": c.title(),
                                "type": "bar",
                                "stack": "total",
                                "itemStyle": {"color": palettes[i][0]},
                                "label": {"show": True},
                                "emphasis": {"focus": "series"},
                                "data": valuesMean,
                                "label": {"show": False}
                                }        

                # Convert the dictionary to a JSON string with a custom serializer
                circleMetric = dictToJs(circleMetric)
                circleMetrics.append(circleMetric)
                
            circleMetricsString = ",".join(circleMetrics)
            
            codeSnippetJS = """
            // Initialize the echarts instance based on the prepared dom
            var myChart = echarts.init(document.getElementById('main'));

            // Apply Montserrat font to ECharts container
            document.getElementById('main').style.fontFamily = 'Montserrat, sans-serif';
            
            var option = {
                title: {
                    text: 'Stacked """+option.title()+""" (%) Of All Circles Over Time',  // Add this line to set the title
                    left: '0%',  // Optional: Align the title
                    top: 'top',  // Optional: Set the position of the title
                    textStyle: {  // Optional: Style the title
                        color: '#000',
                        fontFamily: 'Montserrat',
                        fontSize: 12
                    }                
                },
                tooltip: {
                    trigger: 'axis',
                    axisPointer: {
                    // Use axis to trigger tooltip
                    type: 'shadow' // 'shadow' as default; can also be 'line' or 'shadow'
                    },
                    textStyle: { 
                        fontFamily: 'Montserrat'
                    }
                },
                legend: {
                    top: 20, //use this to move legend below title
                    itemGap: 50, // space between legend items
                    textStyle: {  // Added textStyle attribute here to set the font for legend text
                        fontFamily: 'Montserrat',
                        fontSize: 10   
                    }
                },
                grid: {
                    left: '3%',
                    right: '4%',
                    bottom: '3%',
                    top: '30%',  // Increased the space between the grid and the title
                    containLabel: true
                },
                xAxis: {
                    type: 'category',
                    data: """+str(monthsRange)+""",
                    axisLabel: {  // Added axisLabel attribute here to set the font for axis labels
                        fontFamily: 'Montserrat'
                    }
                },
                yAxis: {
                    type: 'value',
                    name: '"""+option.title()+"""',
                    nameLocation: 'center',
                    nameGap: 50,
                    nameTextStyle: {
                        color: '#000',
                        fontSize: 14,
                        fontFamily: 'Montserrat'  // Added fontFamily here
                    },
                    axisLabel: {  // Added axisLabel attribute here to set the font for axis labels
                        fontFamily: 'Montserrat'
                    }
                },
                series: [ """+circleMetricsString+"""
                ]
            };
            
            var myChart = echarts.init(document.getElementById('main'));
            myChart.setOption(option);
            """
            
        if plot == 'line':
            option = 'performance'
            circleMetrics = []
            for i,c in enumerate(circles):
                # average values for a period of time
                kpis = dfTemp[dfTemp.circle == c].kpi.unique().tolist()
                collectValues = []
                for k in kpis:
                    periodicity = dfTemp.loc[(dfTemp.circle == c)
                                             &(df.kpi == k),'periodicity'].unique().tolist()[0]
                    if periodicity == 'month':
                        dfTempLocal = dfTemp.loc[(df.kpi == k)]
                        values = calculateMetric(dfTempLocal,
                                                 dfPerformance.loc[dfPerformance.kpi == k],
                                                 option)
                        collectValues.append(values)
                    
                    elif periodicity == 'quarter':
                        # for now only works with circles that are only quarterly
                        dfTempLocal = dfTemp.loc[(df.kpi == k)&
                                            (df.date.dt.month.isin([3,6,9,12]))]
                        
                        values = calculateMetric(dfTempLocal,
                                                 dfPerformance.loc[dfPerformance.kpi == k],
                                                 option)                     
                        # use if needed to adapt to monthly plots
                        values = [[v]*3 for v in values]
                        values = [item for sublist in values for item in sublist]
                        collectValues.append(values)
                    
                    elif periodicity == 'year':
                        dfTempLocal = dfTemp.loc[(df.kpi == k)&
                                    (df.date.dt.month.isin([12]))]
                        if dfTempLocal.empty==True:
                            dfTempLocal = dfTemp.loc[(df.kpi == k)&
                            (df.date.dt.month.isin([month]))]

                        values = calculateMetric(dfTempLocal,
                                                 dfPerformance.loc[dfPerformance.kpi == k],
                                                 option)
                        values = [values]*month
                        values = [item for sublist in values for item in sublist]
                        collectValues.append(values)
                
                # combine all kpis in a mean value per circle        
                valuesMean = list(np.mean(np.transpose(collectValues),axis=1).round(2))
                # print(valuesMean)
                circleMetric = {
                                "name": c.title(),
                                "type": "line",
                                "smooth": "True",
                                "itemStyle": {"color": palettes[i][0]},
                                "label": {"show": True},
                                "emphasis": {"focus": "series"},
                                "data": valuesMean,
                                "label": {"show": False}
                                }        

                # Convert the dictionary to a JSON string with a custom serializer
                circleMetric = dictToJs(circleMetric)
                circleMetrics.append(circleMetric)
                
            circleMetricsString = ",".join(circleMetrics)
            
            codeSnippetJS = """
            // Initialize the echarts instance based on the prepared dom
            var myChart = echarts.init(document.getElementById('main'));

            // Apply Montserrat font to ECharts container
            document.getElementById('main').style.fontFamily = 'Montserrat, sans-serif';
            
            var option = {
                title: {
                    text: 'Historical """+option.title()+""" (%) Per Circle Over Time',  // Add this line to set the title
                    left: '0%',  // Optional: Align the title
                    top: 'top',  // Optional: Set the position of the title
                    textStyle: {  // Optional: Style the title
                        color: '#000',
                        fontFamily: 'Montserrat',
                        fontSize: 12
                    }                
                },
                tooltip: {
                    trigger: 'axis',
                    axisPointer: {
                    // Use axis to trigger tooltip
                    type: 'shadow' // 'shadow' as default; can also be 'line' or 'shadow'
                    },
                    textStyle: { 
                        fontFamily: 'Montserrat'
                    }
                },
                legend: {
                    top: 20, //use this to move legend below title
                    itemGap: 50, // space between legend items
                    textStyle: {  // Added textStyle attribute here to set the font for legend text
                        fontFamily: 'Montserrat',
                        fontSize: 10   
                    }
                },
                grid: {
                    left: '3%',
                    right: '4%',
                    bottom: '3%',
                    top: '30%',  // Increased the space between the grid and the title
                    containLabel: true
                },
                xAxis: {
                    type: 'category',
                    data: """+str(monthsRange)+""",
                    axisLabel: {  // Added axisLabel attribute here to set the font for axis labels
                        fontFamily: 'Montserrat'
                    }
                },
                yAxis: {
                    type: 'value',
                    name: '"""+option.title()+"""',
                    nameLocation: 'center',
                    nameGap: 50,
                    nameTextStyle: {
                        color: '#000',
                        fontSize: 14,
                        fontFamily: 'Montserrat'  // Added fontFamily here
                    },
                    axisLabel: {  // Added axisLabel attribute here to set the font for axis labels
                        fontFamily: 'Montserrat'
                    }
                },
                series: [ """+circleMetricsString+"""
                ]
            };
            
            var myChart = echarts.init(document.getElementById('main'));
            myChart.setOption(option);
            """
            
        elif plot == 'circular':
            # cumulative progress per circle
            circleMetrics = []
            for i,c in enumerate(circles):
                # take the last value for each circle because cumulative
                kpis = dfTemp.loc[(dfTemp.circle == c)].kpi.unique().tolist()
                values = [calculateMetric(dfTemp.loc[(dfTemp.kpi == k)],
                                        dfPerformance.loc[(dfPerformance.kpi == k)],
                                        option)[-1] for k in kpis]
                value = np.mean(values)
                
                circleMetric = {
                    "value": value+0.0001, # add 0.0001 to avoid 0.0, otherwise there will be a bug with the fonts
                    "itemStyle": {"color": palettes[i][0]}
                            }        

                # Convert the dictionary to a JSON string with a custom serializer
                circleMetric = dictToJs(circleMetric)
                circleMetrics.append(circleMetric)
                
            circleMetricsString = ",".join(circleMetrics)
            
            codeSnippetJS = """
            // Initialize the echarts instance based on the prepared dom
            var myChart = echarts.init(document.getElementById('main'));

            // Apply Montserrat font to ECharts container
            document.getElementById('main').style.fontFamily = 'Montserrat, sans-serif';
            
            var option = {
                title: [
                    {
                        text: 'Average """+option.title()+""" (%) Per Circle',  // Add this line to set the title
                        left: '0%',  // Optional: Align the title
                        top: 'top',  // Optional: Set the position of the title
                        textStyle: {
                            color: '#000',
                            fontFamily: 'Montserrat',
                            fontSize: 12
                        },
                        left: '0%'
                    }
                ],
                series: {
                    name: 'Progress',
                    type: 'bar',
                    data: ["""+circleMetricsString+"""
                    ],
                    coordinateSystem: 'polar',
                    label: {
                        show: false,
                        position: 'middle',
                        formatter: '{b}: {c}',
                        textStyle: {
                            fontFamily: 'Montserrat'
                        }
                    }
                },
                tooltip: { // formated here to show round values at display
                    trigger: 'axis',
                    axisPointer: {
                        type: 'shadow'
                    },
                    textStyle: { 
                        fontFamily: 'Montserrat'
                    },
                    formatter: function(params) {
                        var tip = params[0].axisValueLabel + '<br>';
                        params.forEach(function(param) {
                            tip += param.marker + ' ' + param.seriesName + ': ' + Math.round(param.value) + '<br>';
                        });
                        return tip;
                    }
                },
                polar: {
                    radius: [30, '80%']
                },
                angleAxis: {
                    max: 100,
                    startAngle: 90,
                    axisLabel: {
                        textStyle: {
                            fontFamily: 'Montserrat'
                        }
                    }
                },
                radiusAxis: {
                    type: 'category',
                    data: """+str(circles)+""", // introduce names for circles
                    show: false, // hide labels in front of bars, too messy
                    axisLabel: {
                        textStyle: {
                            fontFamily: 'Montserrat'
                        }
                    }
                }
            };

            var myChart = echarts.init(document.getElementById('main'));
            myChart.setOption(option);    
            """
            
        elif plot == 'donut':
            # average of progress for all circles
            # take the last month value in the range
            circleMetrics = []
            for i,c in enumerate(circles):
                # take the last value for each circle because cumulative
                kpis = dfTemp.loc[(dfTemp.circle == c)].kpi.unique().tolist()
                values = [calculateMetric(dfTemp.loc[(dfTemp.kpi == k)],
                                        dfPerformance.loc[(dfPerformance.kpi == k)],
                                        option)[-1] for k in kpis]
                circleMetrics.append(np.mean(values))
            
            valuesMean = np.mean(circleMetrics).round(0)
            # baseline = 0
            target = 100
            
            codeSnippetJS = """
            // Initialize the echarts instance based on the prepared dom
            var myChart = echarts.init(document.getElementById('main'));

            // Apply Montserrat font to ECharts container
            document.getElementById('main').style.fontFamily = 'Montserrat, sans-serif';
    
            var value = """+str(valuesMean)+""";  // This is the value you want to display

            var option = {
                title: [
                    {
                        text: 'Average """+option.title()+""" (%) For Projuventute, All Circles Included',
                        left: '0%',  
                        top: 'top',  
                        textStyle: {
                            color: '#000',
                            fontFamily: 'Montserrat',
                            fontSize: 12
                        }
                    }
                ],
                tooltip: {
                    trigger: 'item',
                    textStyle: { 
                        fontFamily: 'Montserrat'
                    },
                    formatter: function(params) {
                        return params.name + ': ' + params.value.toFixed(1) + '%';
                    }
                },
                series: [
                    {
                        name: '"""+option.title()+"""',
                        type: 'pie',
                        radius: ['50%', '80%'],
                        data: [
                            {value: """+str(valuesMean)+""", name: '"""+option.title()+"""', itemStyle: {color: '#E5A200'}},
                            // This data entry will represent the unfilled portion of the donut
                            {value: """""+str(target-valuesMean)+""", name: 'Remaining', itemStyle: {color: '#F0EEEB'}}
                        ],
                        label: {
                            show: true,
                            position: 'center',
                            formatter: function() {
                                return value.toFixed(1) + '%';  // Updated to round the values displayed in the center
                            },
                            fontSize: 30,
                            fontWeight: 'bolder',
                            fontFamily: 'Montserrat',
                            color: '#FBBB21'
                        }
                    }
                ]
            };            
            myChart.setOption(option);   
            """
        
        elif plot == 'radial':
            # select metric from year start until selected month, all circles
            dfTempLocal = df.loc[
                (df.date <= pd.to_datetime(str(year)+'-'+str(month)+'-01'))&
                (df.date >= pd.to_datetime(str(year)+'-01-01'))]
            circles = dfTempLocal.circle.unique().tolist()
            
            kpisAll = []
            kpiMetricsAll = []
            for i,c in enumerate(circles):
                kpisCircle = dfTemp.loc[dfTemp.circle == c].kpi.unique().tolist()
                # kpisAll.append(kpisCircle)
                kpiMetrics = []
                for j,k in enumerate(kpisCircle):
                    value = calculateMetric(dfTempLocal.loc[(dfTempLocal.kpi == k)],
                                            df.loc[(df.kpi == k)],
                                            option)[-1]+ 0.0001
                    kpiMetric = {"value": value, "name": k, "itemStyle": {"color": palettes[i][j]}}
                    kpisAll.append("C"+str(i+1)+": "+k)
                    
                    # Convert the dictionary to a JSON string with a custom serializer
                    kpiMetric = dictToJs(kpiMetric)
                    kpiMetrics.append(kpiMetric)
                kpiMetricsAll.append(kpiMetrics)
            
            # kpisAll = [item for sublist in kpisAll for item in sublist] #flatten list if needed
            # format string with line breaks for labels
            kpisAll = [breakString(s.replace('/', ' / '), 30) for s in kpisAll]
            kpiMetricsAll = [item for sublist in kpiMetricsAll for item in sublist]
            kpiMetricsString = ",".join(kpiMetricsAll)
            
            codeSnippetJS = """
            // Initialize the echarts instance based on the prepared dom
            var myChart = echarts.init(document.getElementById('main'));

            // Apply Montserrat font to ECharts container
            document.getElementById('main').style.fontFamily = 'Montserrat, sans-serif';

            // Configure chart options
            option = {
                textStyle: {
                    fontFamily: 'Montserrat'
                },
                title: [
                    {
                        text: 'Collective """+option.title()+""" (%), All Circles And KPIs Included',
                        left: '0%',  // Adjusted: Align the title to the left
                        top: 'top',
                        textStyle: {
                            color: '#000',
                            fontFamily: 'Montserrat',  // Adjusted: Set the font to Montserrat
                            fontSize: 12  // Adjusted: Set the font size
                        }
                    }
                ],
                legend: {
                    show: true,
                    top: 35,  // Adjust this value based on the actual height of your title
                    type: 'scroll',
                    padding: [5, 10],
                    orient: 'horizontal',  // Change the orientation to horizontal
                    left: 'center',        // Center the legend
                    textStyle: {
                        fontSize: 10  // Adjusted the legend font size here
                    },
                },
                tooltip: {
                    trigger: 'item',
                    textStyle: { 
                        fontFamily: 'Montserrat'
                    },
                    formatter: function(params) {
                        return params.name + '<br>' + params.marker + ' ' + Math.round(params.value);
                    }
                },
                toolbox: {
                    show: false,
                    feature: {
                    mark: { show: false },
                    dataView: { show: true, readOnly: false },
                    restore: { show: true },
                    saveAsImage: { show: true }
                    }
                },
                series: [
                    {
                    name: 'KPIs Progress',
                    type: 'pie',
                    radius: [20, 150],
                    center: ['50%', '60%'],
                    roseType: 'area',
                    label: {
                        position: 'outside',  // Place labels outside the pie chart
                        formatter: '{b}: {d}%',  // Display the name and percentage
                        color: '#000', // Set label text color
                        fontSize: 0 //hide for now
                    },
                    labelLine: {
                        show: false,  // Display the label line
                        length: 15,  // Length of the line from the pie to the label
                        length2: 10  // Length from the end of the first line to the label
                    },
                    itemStyle: {
                        borderRadius: 0
                    },
                    data: ["""+kpiMetricsString+"""],
                    },

                ]
            };
            myChart.setOption(option);
            """
        
        if plot == 'streamgraph':
            # cumulative progress all circles per kpi with time vector
            dataAllKpis = [] 
            allKpis = []
            coloursAllKpis = [] 
            # define all dates from january until selected month
            dates = dfTemp.sort_values('date').loc[:,'date'].unique().tolist()
            dates = [d.strftime('%Y-%m') for d in dates]
         
            for i,c in enumerate(circles):
                
                kpis = dfTemp.loc[dfTemp.circle == c].kpi.unique().tolist()
                for j,k in enumerate(kpis):
                    periodicity = dfTemp.loc[(dfTemp.circle == c)&
                                             (df.kpi == k),'periodicity'].unique().tolist()[0]
                    
                    if periodicity == 'month':
                        dfTempLocal = dfTemp.loc[(df.kpi == k)]
                        values = calculateMetric(dfTempLocal,
                                                 dfPerformance.loc[dfPerformance.kpi == k],
                                                 option)
                    
                    elif periodicity == 'quarter':
                        # for now only works with circles that are only quarterly
                        dfTempLocal = dfTemp.loc[(df.kpi == k)&
                                            (df.date.dt.month.isin([3,6,9,12]))]
                        
                        values = calculateMetric(dfTempLocal,
                                                 dfPerformance.loc[dfPerformance.kpi == k],
                                                 option)                       
                        # use if needed to adapt to monthly plots
                        values = [[v]*3 for v in values]
                        values = [item for sublist in values for item in sublist]
                    
                    elif periodicity == 'year':
                        dfTempLocal = dfTemp.loc[(df.kpi == k)&
                                    (df.date.dt.month.isin([12]))]
                        
                        if dfTempLocal.empty==True:
                            dfTempLocal = dfTemp.loc[(df.kpi == k)&
                            (df.date.dt.month.isin([month]))]
                        
                        values = calculateMetric(dfTempLocal,
                                                 dfPerformance.loc[dfPerformance.kpi == k],
                                                 option)
                        # use if needed to adapt to monthly plots
                        values = [values]*month
                        values = [item for sublist in values for item in sublist]
                    
                    data = list(zip(dates, values, [k]*len(dates)))
                    data = [list(d) for d in data]
                    
                    allKpis.append(k)
                    dataAllKpis.append(data)
                    coloursAllKpis.append(palettes[i][j])
            
            # flatten data        
            dataAllKpis = [item for sublist in dataAllKpis for item in sublist]
            
            codeSnippetJS = """
            // Initialize the echarts instance based on the prepared dom
            var myChart = echarts.init(document.getElementById('main'));

            // Apply Montserrat font to ECharts container
            document.getElementById('main').style.fontFamily = 'Montserrat, sans-serif';

            // Configure chart options
            option = {
                title: {
                    text: 'Collective """+option.title()+""" (%) Over Time, All Circles And KPIs Included',
                    left: '0%',
                    top: 'top',
                    textStyle: {
                        color: '#000',
                        fontFamily: 'Montserrat',
                        fontSize: 12
                    }
                },
                tooltip: {
                    trigger: 'axis',
                    textStyle: { 
                        fontFamily: 'Montserrat', 
                        fontSize: 12 
                    },
                    axisPointer: {
                    type: 'line',
                    lineStyle: {
                        color: 'rgba(0,0,0,0.2)',
                        width: 1,
                        type: 'solid'
                    }
                    }
                },

                legend: {
                    show: true,
                    top: 40,  // Adjust this value based on the actual height of your title
                    type: 'scroll',
                    padding: [5, 10],
                    orient: 'horizontal',  // Change the orientation to horizontal
                    left: 'center',        // Center the legend
                    textStyle: {
                        fontSize: 10  // Adjusted the legend font size here
                    },
                    data: """+str(allKpis)+""",
                },
                singleAxis: {
                    top: 50,
                    bottom: 50,
                    axisTick: {},
                    axisLabel: {
                        textStyle: {
                            fontFamily: 'Montserrat', 
                            fontSize: 10 
                        }
                    },
                    type: 'time',
                    axisPointer: {
                    animation: true,
                    label: {
                        show: true,
                        textStyle: {
                            fontFamily: 'Montserrat', 
                            fontSize: 10 
                        }
                    }
                    },
                    splitLine: {
                    show: true,
                    lineStyle: {
                        type: 'dashed',
                        opacity: 0.2
                    }
                    }
                },
                series: [
                    {
                    type: 'themeRiver',
                    color: """+str(coloursAllKpis)+""",
                    itemStyle: {
                        emphasis: {
                            shadowBlur: 10,
                            shadowColor: 'rgba(0, 0, 0, 0.8)'
                        }
                    },
                    data: """+str(dataAllKpis)+""",
                    label: {
                        show: false,
                        textStyle: {
                            fontFamily: 'Montserrat', 
                            fontSize: 10 
                        }
                    }
                    }
                ]
                };
                myChart.setOption(option);
                """
   
        launchHtml(codeSnippetJS,option+"_"+plot, circle ,year)
        
    else:
        print('there is no values with this combination')

In [1075]:
circles

['Digital',
 'Fundraising',
 'HR',
 'Program',
 'Programs - Children - Counceling',
 'Programs - Parents -Online']

In [678]:
year = 2023
month = 12
for c in circles:
    plotActualDataJS(   df=df,
                        circle=c,
                        year=year,
                        month=month)

In [24]:
for c in circles:
    for t in ['radial']:
        snippet = plotMetricsJS( df=df,
                                    circle=c,
                                    year=2023,
                                    month=12,
                                    option='progress',#'performance'
                                    plot=t
                                    )

In [681]:
for t in ['circular','donut','radial','bar','streamgraph','line']:
    plotMetricsGatekeeperJS(df=df,
                            year=year,
                            month=12,
                            option = 'progress', #'performance' >>> what should be displayed
                            plot = t,#'bar','circular','donut','radial','heatmap','streamgraph'
                            monthList=monthList)

In [20]:
plotMetricsGatekeeperJS(df=df,
                        year=2023,
                        month=12,
                        option = 'progress', #'performance' >>> what should be displayed
                        plot = 'radial',#'bar','circular','donut','radial','streamgraph'
                        monthList=monthList)